In [2]:
#properties final


import pandas as pd
from datetime import datetime


contacts_file = r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\final_combined_contacts_data.xlsx'
owner_contacts_file = r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\owner_contacts.xlsx'
supplier_contacts_file = r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\supplier_contacts.xlsx'
property_images_file = r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\property_images.xlsx'
properties_data_file = r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\final_combined_properties_data.xlsx'
owner_folios_file = r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\owner_folios.xlsx'


contacts_df = pd.read_excel(contacts_file)
owner_contacts_df = pd.read_excel(owner_contacts_file)
supplier_contacts_df = pd.read_excel(supplier_contacts_file)
property_images_df = pd.read_excel(property_images_file)
properties_df = pd.read_excel(properties_data_file)
owner_folios_df = pd.read_excel(owner_folios_file)


property_type_mapping = {
    'Townhouse': 1,
    'Apartment': 2,
    'House': 3,
    'Unit': 4,
    'Studio': 5
}


def map_primary_type(property_type):
    return 'Commercial' if property_type == 'Other' else 'Residential'


def extract_attributes(attr_str):
    try:
        attributes = attr_str.split()
        if len(attributes) >= 3:
            bedroom = int(attributes[0])
            bathroom = int(attributes[1])
            car_space = int(attributes[2])
        else:
            bedroom = bathroom = car_space = None
    except:
        bedroom = bathroom = car_space = None
    return bedroom, bathroom, car_space


properties_data = []
current_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Starting the id from 200001
start_id = 200001

for index, row in properties_df.iterrows():
    bedroom, bathroom, car_space = extract_attributes(row['Property Attributes'])
    
    
    property_type = property_type_mapping.get(row['Property Type '], None)
    
    
    primary_type = map_primary_type(row['Property Type '])
    
    
    strata_manager_id = supplier_contacts_df[supplier_contacts_df['reference'] == row['Reference']]['id'].values
    strata_manager_id = strata_manager_id[0] if len(strata_manager_id) > 0 else None
    
    
    routine_inspections_frequency_type = contacts_df[contacts_df['Inner_Property'] == row['Reference']]['Inner_Period'].values
    routine_inspections_frequency_type = routine_inspections_frequency_type[0] if len(routine_inspections_frequency_type) > 0 else None
    
    
    first_routine_frequency_type = contacts_df[contacts_df['Inner_Property'] == row['Reference']]['Inner_Period'].values
    first_routine_frequency_type = first_routine_frequency_type[0] if len(first_routine_frequency_type) > 0 else None
    
    
    property_image = property_images_df[property_images_df['property_id'] == row['properties_row']]['property_image'].values
    property_image = property_image[0] if len(property_image) > 0 else None
    
    
    owner_contact_id = owner_contacts_df[owner_contacts_df['reference'] == row['Owner']]['id'].values
    owner_contact_id = owner_contact_id[0] if len(owner_contact_id) > 0 else 'NULL'
    
    
    owner_folio_id = owner_folios_df[owner_folios_df['property_id'] == row['properties_row']]['Id'].values
    owner_folio_id = owner_folio_id[0] if len(owner_folio_id) > 0 else 'NULL'
    
    
    owner = 1 if pd.notna(row['Owner']) else 0
    tenant = 1 if pd.notna(row['Tenant']) else 0
    
    
    routine_inspection_due_date = row['Property Inspection due']
    if pd.notna(routine_inspection_due_date):
        routine_inspection_due_date = pd.to_datetime(routine_inspection_due_date).strftime('%Y-%m-%d')
    else:
        routine_inspection_due_date = None
    
    
    properties_data.append([
        start_id + index,  # Auto incremental ID starting from 200001
        row['Reference'],  # Reference
        row['manager_id'],  # Manager ID
        None,  # Location (NULL for now)
        property_type,  # Property Type
        primary_type,  # Primary Type
        None,  # Description (NULL)
        bathroom,  # Bathroom
        bedroom,  # Bedroom
        car_space,  # Car Space
        row['Floor area'],  # Floor Area
        None,  # Floor Size (NULL)
        row['Land area'],  # Land Area (same as Floor Area)
        None,  # Land Size (NULL)
        row['Property Key number'],  # Key Number
        strata_manager_id,  # Strata Manager ID
        None,  # Routine Inspections Frequency (NULL)
        routine_inspections_frequency_type,  # Routine Inspections Frequency Type
        None,  # First Routine (NULL)
        first_routine_frequency_type,  # First Routine Frequency Type
        routine_inspection_due_date,  # Routine Inspection Due Date in YYYY-MM-DD format
        None,  # Note (NULL)
        property_image,  # Property Image
        owner,  # Owner (1 if present, 0 if not)
        tenant,  # Tenant (1 if present, 0 if not)
        1,  # Company ID
        current_timestamp,  # Created At
        current_timestamp,  # Updated At
        'Active',  # Status
        None,  # YouTube Link (NULL)
        None,  # VR Link (NULL)
        None,  # Subject (NULL)
        owner_folio_id,  # Owner Folio ID ('NULL' if empty)
        owner_contact_id  # Owner Contact ID ('NULL' if empty)
    ])


columns = [
    'id', 'reference', 'manager_id', 'location', 'property_type', 'primary_type', 'description',
    'bathroom', 'bedroom', 'car_space', 'floor_area', 'floor_size', 'land_area', 'land_size',
    'key_number', 'strata_manager_id', 'routine_inspections_frequency', 'routine_inspections_frequency_type',
    'first_routine', 'first_routine_frequency_type', 'routine_inspection_due_date', 'note', 'property_image',
    'owner', 'tenant', 'company_id', 'created_at', 'updated_at', 'status', 'youtube_link', 'vr_link',
    'subject', 'owner_folio_id', 'owner_contact_id'
]
properties_df_new = pd.DataFrame(properties_data, columns=columns)


output_file = r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\properties.xlsx'
properties_df_new.to_excel(output_file, index=False)

print(f"properties.xlsx file has been created successfully at {output_file}.")


properties.xlsx file has been created successfully at C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\properties.xlsx.
